# Woman Dress Amazon Web Scraping Project

In [1]:
!pip install selenium

In [2]:
from selenium import webdriver

webdriver.Chrome ใช้ในการเปิด google chrome

In [3]:
driver = webdriver.Chrome(executable_path=r'C:\Users\User\Desktop\webscraper project\chromedriver')

<ipython-input-3-a3a955e4ef4f>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\Users\User\Desktop\webscraper project\chromedriver')


.get() ใช้ในการเปิดเว็บไซต์

In [4]:
driver.get('https://www.amazon.com/')

.find_element("xpath", ' ') ใช้ในการหาตำแหน่งที่เราสนใจในเว็บไซต์ เช่น ในที่นี้เราอยากกรอกข้อมูลเพื่อค้นหาสินค้าใน amazon เราจึงต้องเข้าไปดู html code เพื่อหาตำแหน่งของช่องว่างที่ให้เรากรอกข้อมูลเพื่อค้นหาสินค้า ซึ่งจะได้ตาม code ด้านล่างเป็นดังนี้ (ถ้าหากเจอ shadow root คำสั่งนี้ไม่สามารถใช้ได้ จำเป็นต้องใช้ .execute_script เพื่อใช้ระบุตำแหน่งแทน) 

In [5]:
option = driver.find_element("xpath", '/html/body/div[1]/div[2]/div[2]/div[1]/div[6]/div')

.click() ใช้ในการสั่ง click เว็บไซต์

In [6]:
option.click()

ต่อมาเราจะทำการเรียก script เพื่อใช้ในการย่อขนาดของหน้าเว็บไซท์ เพื่อให้มันอ่าน code ของหน้าเว็บไซต์ทั้งหมด ที่ active อยู่

In [22]:
driver.execute_script("document.body.style.zoom = '100%'")

.page_source ดึง html ของ browser ที่เปิดอยู่ (ทำให้ data เป็น text)

In [18]:
data = driver.page_source

In [19]:
!pip install bs4

In [20]:
import bs4

เรียกใช้ BeautifulSoup เพื่อทำให้ก้อน html ที่เราดึงมาจาก .page_source กลายเป็น object เพื่อให้เราสามารถดึง html tags ที่เราต้องการได้ ซึ่งในที่นี้เราต้องการชื่อสินค้า ราคา rating และจำนวนผู้ให้คะแนน

In [134]:
soup = bs4.BeautifulSoup(data)

In [40]:
all_item = soup.find_all('div', {'class':'a-section a-spacing-small puis-padding-left-small puis-padding-right-small'})

In [234]:
product_name = []
price = []
rating = []
amount_votes = []

for item in all_item:
    all_product_name = item.find('span', {'class':'a-size-base-plus a-color-base a-text-normal'})
    product_name.append(all_product_name.text)
    all_price = item.find('span', {'aria-hidden':'true'})
    price.append(all_price.text)
    all_rating = item.find('span', {'class':'a-icon-alt'})
    if all_rating == None:
        all_rating = None
    else:
        all_rating = all_rating.text
    rating.append(all_rating)
    all_amount_votes = item.find('span', {'class':'a-size-base s-underline-text'})
    if all_amount_votes == None:
        all_amount_votes = None
    else:
        all_amount_votes = all_amount_votes.text
    amount_votes.append(all_amount_votes)

In [98]:
import pandas as pd

จากนั้นนำ data ทั้งหมดที่เราขูดออกมาจากหน้าเว็บไซต์ดังกล่าวมาทำ DataFrame()

In [109]:
women_dress = pd.DataFrame()

In [235]:
women_dress['product_name'] = product_name
women_dress['price'] = price
women_dress['rating'] = rating
women_dress['amount_votes'] = amount_votes

In [237]:
women_dress.dtypes

product_name    object
price           object
rating          object
amount_votes    object
dtype: object

เรียกใช้ .str.replace() เพื่อทำการแทนที่สิ่งที่เราต้องการหรือ remove สิ่งที่ไม่ต้องการออกจาก data และเปลี่ยน data type โดยการเรียกใช้ pd.to_numeric() และกำหนด downcast = " " 
#### ถ้าเราต้องการเปลี่ยน string เป็น:
- float --> downcast = "Float"
- int --> downcast = "integer"

In [238]:
women_dress['price'] = women_dress['price'].str.replace('$','')
women_dress['price'] = pd.to_numeric(women_dress['price'], downcast="float")
women_dress.dtypes

product_name     object
price           float32
rating           object
amount_votes     object
dtype: object

In [245]:
women_dress['rating'] = women_dress['rating'].str.replace('out of', '/')
women_dress['rating'] = women_dress['rating'].str.replace('stars', '')

In [239]:
women_dress['amount_votes'] = women_dress['amount_votes'].str.replace(',','')

ต่อมาจะทำการเก็บ data ทั้งหมดลงใน csv file

In [249]:
women_dress.to_csv('WomenDressAmazonDataset.csv',index=False) 